In [201]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report


# CRISP
## Business understanding:
- The first phase is to understand the business problem that needs to be solved.
- What is the goal of the analysis?
- What are the requirements and constraints? What is the expected outcome?

## Data understanding:
 - The second phase is to collect and explore the data.
 - What data is available?
 - What are the characteristics of the data (variable types, value distributions etc.)?
 - Are there any quality issues with the data (missing values, outliers, nonsensical values)?

## Data preparation:
 - The third phase is to preprocess the data.
 - This includes cleaning the data, transforming the data, and selecting the relevant features.
 - These steps should be documented in such detail that they can be reproduced later.

## Modeling:
 - The fourth phase is to choose a machine learning method and train the model.
 - This phase also includes the validation of the model.
 - Documentation needs include:
   - which method was used, which parameters were used, what was the performance of the model?

## Evaluation:
 - The fifth phase is to evaluate the model.
 - How well does the model perform?
 - Does it meet the business requirements?

## Deployment:
 - The final phase is to deploy the model.
 - How will the model be used in practice?
 - How will the results be communicated?
 - This phase may involve creating a recommendation of how to use the model in practice, or what to do next.


## Phase 1: Business Understanding
- The goal is to reliably predict if website is a phishing site based on the information about the site using UCI Machine Learning Repository: Phishing Websites Data Set.

## Phase 2: Data understanding


In [202]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
phishing_websites = fetch_ucirepo(id=327)

# data (as pandas dataframes)
X = phishing_websites.data.features
y = phishing_websites.data.targets

print(phishing_websites.variables.name)

0              having_ip_address
1                     url_length
2             shortining_service
3               having_at_symbol
4       double_slash_redirecting
5                  prefix_suffix
6              having_sub_domain
7                 sslfinal_state
8     domain_registration_length
9                        favicon
10                          port
11                   https_token
12                   request_url
13                 url_of_anchor
14                 links_in_tags
15                           sfh
16           submitting_to_email
17                  abnormal_url
18                      redirect
19                  on_mouseover
20                    rightclick
21                   popupwindow
22                        iframe
23                 age_of_domain
24                     dnsrecord
25                   web_traffic
26                     page_rank
27                  google_index
28        links_pointing_to_page
29            statistical_report
30        

## Phase 3: Data Preparation

The third phase is to preprocess the data. We remove missing values and transform categorical variables into numeric form to make the dataset ready for machine learning models.

By removing missing values to ensures that we work only with complete data.

By selecting relevant features, we improve model performance, reduce overfitting, and make the model easier to interpret.


 By selecting relevant features, we improve model performance, reduce overfitting, and make the model easier to interpret.

### Steps 

#### Removing Missing Values:

Using `dropna()` from Pandas. This ensures that we work only with complete data.

In [203]:
# Data Cleaning
# - Remove missing values to ensure data consistency
X = X.dropna()  # Remove rows with missing values


Transforming Target Column (`y`): use the `replace()` method in Pandas to replace `-1` with `0` and leave `1` unchanged.

In [204]:
# Convert 'result' column to numeric: 0 = Legal, 1 = Phishing site
y = y.replace({-1: 0, 1: 1})  # Replace -1 with 0 and 1 remains as 1

#### Feature Selection:

We choose features based on what we know about the field or how important each feature is for predicting the target variable.

Using the selected feature list, we create a new dataframe with only these columns.

In [211]:
# Select important features
selected_features = [
    'url_length', 'shortining_service', 'having_at_symbol', 'double_slash_redirecting', 
    'prefix_suffix', 'having_sub_domain', 'sslfinal_state', 'abnormal_url', 'https_token', 'redirect', 'url_of_anchor', 'iframe', 'request_url', 'submitting_to_email'
]

#### Plitting the Data:

 To evaluate the model's performance on unseen data, we split the data into two subsets: 
    - Training data (80%) for training the model.
    - Test data (20%) for evaluating how well the model generalizes to new data.
    
  We use `train_test_split()` from scikit-learn to divide the data into training and test sets. 

In [212]:
# Create X with only selected features
X_selected = X[selected_features]

# Data Splitting
# Split the data: 80% training data, 20% testing data
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Check the split sizes to ensure it's correct
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(8844, 14) (2211, 14) (8844, 1) (2211, 1)


We use `head()` to print the first few rows of the selected features dataset. Make sure everything looks correct before moving on to training.

In [213]:
print(X_selected.head())  # Print first few rows of the selected features dataset


   url_length  shortining_service  having_at_symbol  double_slash_redirecting  \
0           1                   1                 1                        -1   
1           1                   1                 1                         1   
2           0                   1                 1                         1   
3           0                   1                 1                         1   
4           0                  -1                 1                         1   

   prefix_suffix  having_sub_domain  sslfinal_state  abnormal_url  \
0             -1                 -1              -1            -1   
1             -1                  0               1             1   
2             -1                 -1              -1            -1   
3             -1                 -1              -1             1   
4             -1                  1               1             1   

   https_token  redirect  url_of_anchor  iframe  request_url  \
0           -1         0          

In [208]:
# Phase 4: Modeling
# Choose machine learning method and train the model. This phase also includes the validation of the model.


In [209]:
# Phase 5: Evaluation
# Evaluate the model. How well does the model perform? Does it meet the business requirements?

In [210]:
# Phase 6: Deployment
# Deploy the model. How will the model be used in practice? How will the results be communicated?